In [480]:
import json
from functools import partial
from inspect import getsource
from math import log
from sys import argv
from bisect import bisect_left
import re


def checker(nm,ori):
    flag = False
    for key in ori.keys():
        if ori[key] != nm[key]:
            flag = True
            break
    print('Match!!' if flag == False else 'Error!!')

def test(origin_mapping,idx):
    print(len(idx))
    mark = consecu_int_run_length(idx)
    # check part
    _m = mark
    nm = {}
    for i in range(len(keys)):
        nm[keys[i]] = values[i]
    for i in range(0,len(_m),3):
        for j in range(_m[i+2]):
            nm[keys[_m[i+1]+j]] = values[_m[i]+j]
    checker(nm,origin_mapping)

    sporadic_seg = sporadic_seg_run_length(mark)
    # check part
    _m = sporadic_seg
    nm = {}
    vv = set(values)
    for i in range(len(keys)):
        if keys[i] in values:
            nm[keys[i]] = keys[i]
        #nm[keys[i]] = values[i]
    for i in range(0,len(_m),3):
        if _m[i+2] != 1:
            for j in range(_m[i+2]):
                nm[keys[_m[i+1]+j]] = values[_m[i]+j]
        else:
            length = len(_m[i+1])
            for j in range(length):
                nm[keys[_m[i+1][j]]] = values[_m[i]+j]
    checker(nm,origin_mapping)

def consecu_int_run_length(idx):
    mark = []
    flag = False
    temp = [ 0, 0, 0 ]
    for i in range(len(k)):
        if i != idx[i]:
            if flag == False:
                temp = [ i, idx[i], 1 ]
                flag = True
            elif flag == True and idx[i]-(temp[1]+temp[2]) == 0:
                temp[2] += 1
            elif flag == True:
                mark += temp
                
                #print(temp)
                temp = [ i, idx[i], 1 ]
                flag = True
        else:
            if flag == True:
                mark += temp
                #print(temp)
                flag = False
    return mark

def sporadic_seg_run_length(mark):
    sporadic_seg = [ mark[i:i+3] for i in range(0,len(mark),3)]
    flag = False
    temp = []
    res = []
    for i in range(len(sporadic_seg)):
        if sporadic_seg[i][2] != 1:
            if flag == True:
                res += temp 
                # initialize temp to an empty list, to avoid if temp have something but index is at the end
                temp = []
                flag = False
            res += sporadic_seg[i]
        else:
            # first one
            if flag == False:
                # start index, index list, len
                temp = [sporadic_seg[i][0],[sporadic_seg[i][1]],1]
                flag = True
            else:
                if sporadic_seg[i][0] - sporadic_seg[i-1][0] == 1:
                    temp[1].append(sporadic_seg[i][1]) 
                else:
                    res += temp  
                    temp = [sporadic_seg[i][0],[sporadic_seg[i][1]],1]
    if len(temp) != 0:
        res += temp  
    return res

def classify(origin_mapping):
    same, prefix, infix, postfix, others = [], [], [], [], []
    for k,v in origin_mapping.items():
        index = v.find(k)
        if k == v:
            same += [k]
        elif index != -1:
            if index == 0:
                postfix += [k]
            elif v[index:] == k:
                prefix += [k]
            else:
                infix += [k]
        else:
            others += [k]
    return same, prefix, infix, postfix, others

def remove_same(k,v,same):
    _k, _v = [], []
    same = set(same)
    c = 0
    for i in range(len(k)):
        if k[i] not in same:
            _k += [k[i]]
            c += 1
        if v[i] not in same:
            _v += [v[i]]
    return _k, _v

if __name__ == '__main__':
    case = 8
    map_in_name, script_name = '../cases/case{}.json'.format(case), 'test.py'
    nf = '../input/case{}_in.json'.format(case)
    nf = open(nf, 'r')
    nf = json.load(nf)
    keys, values = sorted(nf[0]), sorted(nf[1])
    
    map_in_file = open(map_in_name)
    origin_mapping = json.load(map_in_file)
    map_in_file.close()
    same, prefix, infix, postfix, others = classify(origin_mapping)
    mmm = sorted(list(origin_mapping.items()),key=lambda x: x[0])

    k,v = zip(*mmm)
    k, v = remove_same(k,v,same)
    keys, values = remove_same(keys,values,same)
    mm = sorted(zip(v,[ i for i in range(len(k)) ]),key=lambda x: x[0])
    print(mm[:10])
    v, idx = zip(*mm)
    idx = list(idx)
    
    
    document = """
import json
import hashlib
from sys import argv

_m = {}
nf, of = argv[1], argv[2]
nf = open(nf, 'r')
nf = json.load(nf)
keys, values = sorted(nf[0]), sorted(nf[1])

nm = {{}}
for i in range(len(keys)):
    nm[keys[i]] = values[i]
for i in range(0,len(_m),3):
    if _m[i+2] != 1:
        for j in range(_m[i+2]):
            nm[keys[_m[i+1]+j]] = values[_m[i]+j]
    else:
        length = len(_m[i+1])
        for j in range(length):
            nm[keys[_m[i+1][j]]] = values[_m[i]+j]
of = open(of, 'w')
of.write(json.dumps(nm))
of.close()
    """.format(str(sporadic_seg))
    script = open(script_name, 'w')
    script.write(document)
    script.close()

[('ga_ad/ga_ad_cd_Detar/ColicRoot_bud[0]/h0/i1/i1/l0', 0), ('ga_ad/ga_ad_cd_Detar/ColicRoot_bud[10]/h0/i1/i1/l0', 1), ('ga_ad/ga_ad_cd_Detar/ColicRoot_bud[11]/h0/i1/i1/l0', 2), ('ga_ad/ga_ad_cd_Detar/ColicRoot_bud[12]/h0/i1/i1/l0', 3), ('ga_ad/ga_ad_cd_Detar/ColicRoot_bud[13]/h0/i1/i1/l0', 4), ('ga_ad/ga_ad_cd_Detar/ColicRoot_bud[14]/h0/i1/i1/l0', 5), ('ga_ad/ga_ad_cd_Detar/ColicRoot_bud[15]/h0/i1/i1/l0', 6), ('ga_ad/ga_ad_cd_Detar/ColicRoot_bud[16]/h0/i1/i1/l0', 7), ('ga_ad/ga_ad_cd_Detar/ColicRoot_bud[17]/h0/i1/i1/l0', 8), ('ga_ad/ga_ad_cd_Detar/ColicRoot_bud[18]/h0/i1/i1/l0', 9)]


In [479]:
document = """import json as a
from sys import argv
w={}
n,o=argv[1:3]
n=open(n,'r')
k,v=a.load(n)
k,v=sorted(k),sorted(v)
m={{}}
for i in range(len(k)):
    m[k[i]]=v[i]
for i in range(0,len(w),3):
    if w[i+2]!=1:
        for j in range(w[i+2]):
            m[k[w[i+1]+j]]=v[w[i]+j]
    else:
        for j in range(len(w[i+1])):
            m[k[w[i+1][j]]]=v[w[i]+j]
o=open(o,'w')
o.write(a.dumps(m))
o.close()"""
print(len(document))

400
